# Libraries

In [1]:
import numpy as np
import cv2
import os

# Load data

In [2]:
train_path='C:/Users/lsreeram/Downloads/_/Step01_Learning/04_ComputerVision/01_POC_Hexagon_AI_PID/03_train'
os.chdir(train_path)

In [3]:
# Load the image and symbol templates
image = cv2.imread('5258-601-A1-0414.jpeg', cv2.IMREAD_COLOR)

In [4]:
template_list = ['template (1).png',
'template (2).png',
'template (3).png',
'template (4).png',
'template (5).png',
'template (6).png']
# Add more template filenames as needed

In [5]:
color_green =(0, 255, 0)
color_pink = (255, 105, 180)
color_deep_pink = (255, 20, 147)# Pink color (RBG format)

In [6]:
sift = cv2.SIFT_create()

In [7]:
# Define a list to store the matched templates & locations
matched_locations = []
matched_templates = []

In [8]:
# Define a threshold for matching score
threshold = 0.66

# NMS

In [9]:
# Iterate over the templates
for template_filename in template_list:
    # Read the template image
    template = cv2.imread(template_filename, cv2.IMREAD_COLOR)

    # Convert the template and image to grayscale
    gray_template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Perform template matching
    result = cv2.matchTemplate(gray_image, gray_template, cv2.TM_CCOEFF_NORMED)
    print(result)

    # Find locations where the matching score exceeds the threshold
    locations = np.where(result >= threshold)

    # Iterate over the locations and store the matched templates
    for loc in zip(*locations[::-1]):
        matched_templates.append(template_filename)
        #print(f"Template '{template_filename}' found at coordinates (x, y): {loc}")
        x, y = loc
        w, h = template.shape[1], template.shape[0]
        matched_locations.append((x, y, x + w, y + h))
        print("Template found at coordinates (x, y):", x, y)

[[-0.08344685 -0.08348399 -0.08346542 ... -0.07529471 -0.06965224
  -0.06732018]
 [-0.08285698 -0.08283842 -0.08283842 ... -0.07518396 -0.06968728
  -0.06746759]
 [-0.08245683 -0.0824754  -0.08249396 ... -0.07529647 -0.06991785
  -0.06779701]
 ...
 [ 0.          0.          0.         ... -0.01179113 -0.01414406
  -0.01631146]
 [ 0.          0.          0.         ... -0.0118186  -0.01416412
  -0.01634643]
 [ 0.          0.          0.         ... -0.01184606 -0.01416412
  -0.01634643]]
Template found at coordinates (x, y): 9519 1673
Template found at coordinates (x, y): 9520 1673
Template found at coordinates (x, y): 9521 1673
Template found at coordinates (x, y): 9522 1673
Template found at coordinates (x, y): 9523 1673
Template found at coordinates (x, y): 9524 1673
Template found at coordinates (x, y): 9808 1673
Template found at coordinates (x, y): 9809 1673
Template found at coordinates (x, y): 9810 1673
Template found at coordinates (x, y): 9811 1673
Template found at coordinate

[[-0.09866457 -0.09867554 -0.09867554 ... -0.09333325 -0.08903948
  -0.08803947]
 [-0.10083109 -0.10085304 -0.10086402 ... -0.09569912 -0.09135436
  -0.09037282]
 [-0.10195944 -0.10198139 -0.10198139 ... -0.09721288 -0.09284788
  -0.09196358]
 ...
 [ 0.          0.          0.         ... -0.02735035 -0.03215773
  -0.03624163]
 [ 0.          0.          0.         ... -0.02735035 -0.03215773
  -0.03624163]
 [ 0.          0.          0.         ... -0.02735035 -0.03213648
  -0.03623236]]
Template found at coordinates (x, y): 9534 1677
Template found at coordinates (x, y): 9535 1677
Template found at coordinates (x, y): 9536 1677
Template found at coordinates (x, y): 9537 1677
Template found at coordinates (x, y): 9538 1677
Template found at coordinates (x, y): 9539 1677
Template found at coordinates (x, y): 9540 1677
Template found at coordinates (x, y): 9541 1677
Template found at coordinates (x, y): 9542 1677
Template found at coordinates (x, y): 9823 1677
Template found at coordinate

[[-0.0913718  -0.0913718  -0.0913718  ... -0.09541213 -0.09494434
  -0.09752254]
 [-0.09045648 -0.09045648 -0.09045648 ... -0.09500696 -0.09474992
  -0.09748675]
 [-0.08998849 -0.08998849 -0.08998849 ... -0.09491887 -0.09481978
  -0.09766427]
 ...
 [-0.02897998 -0.03066176 -0.02875828 ... -0.04007638 -0.04797991
  -0.05531211]
 [-0.02681161 -0.02614843 -0.02587288 ... -0.04006992 -0.04797243
  -0.0553083 ]
 [-0.02420739 -0.02152072 -0.02198781 ... -0.04003153 -0.04795592
  -0.05528869]]
Template found at coordinates (x, y): 4911 4692
Template found at coordinates (x, y): 4912 4692
Template found at coordinates (x, y): 7692 4692
Template found at coordinates (x, y): 7693 4692
Template found at coordinates (x, y): 4909 4693
Template found at coordinates (x, y): 4910 4693
Template found at coordinates (x, y): 4911 4693
Template found at coordinates (x, y): 4912 4693
Template found at coordinates (x, y): 4913 4693
Template found at coordinates (x, y): 4914 4693
Template found at coordinate

# Draw Shapes

In [10]:
# Draw bounding boxes around the matched locations on the image
for (x1, y1, x2, y2) in matched_locations:
    cv2.rectangle(image, (x1,y1), (x2, y2), color_deep_pink,2)

# Show Output

In [14]:
# Get the screen resolution
screen_width, screen_height = 1920, 1080  # Replace with your screen resolution

# Resize the image to the desired resolution
image_output = cv2.resize(image, (screen_width, screen_height))

# Create a named window
cv2.namedWindow('Image', cv2.WINDOW_NORMAL)

# Display the image
cv2.imshow('Image', image_output)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
cv2.imwrite('output1.jpg', image_output)

True